<a href="https://colab.research.google.com/github/ethvedbitdesjan/SummerResearch/blob/main/ViTLarge_Consistent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 65.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 27.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 71.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 44.7 MB/s eta 0:00:00


In [2]:
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 7.5 MB/s eta 0:00:00


In [3]:
from google.colab import drive
drive.mount('content/')

Mounted at content/


In [4]:
%cd content/MyDrive/SummerResearch

/content/content/MyDrive/SummerResearch


In [8]:
import os
len(os.listdir('Dataset_Consistent/valid/Funny_Consistent/'))

172

In [9]:
from datasets import load_dataset

dataset = load_dataset("imagefolder", data_dir="Dataset_Consistent/")

Resolving data files:   0%|          | 0/1203 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/401 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/402 [00:00<?, ?it/s]

Extracting data files: 0it [00:00, ?it/s]

Extracting data files: 0it [00:00, ?it/s]

...

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset imagefolder downloaded and prepared to /root/.cache/huggingface/datasets/imagefolder/default-edd842f627080eff/0.0.0/37fbb85cc714a338bea574ac6c7d0b5be5aff46c1862c1989b20e0771199e93f. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [10]:
dataset

DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 1203
    })
    validation: Dataset({
        features: ['image', 'label'],
        num_rows: 401
    })
    test: Dataset({
        features: ['image', 'label'],
        num_rows: 402
    })
})

In [11]:
# Critical imports
import os
import numpy as np
import pandas as pd
from PIL import Image
import random
import cv2
import copy
import torch
import torchvision
import torchvision.models as models
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torch.optim as optim
from torch.optim import lr_scheduler

In [12]:
import shutil

In [13]:
from transformers import ViTFeatureExtractor

model_name_or_path = 'google/vit-large-patch16-224-in21k'
feature_extractor = ViTFeatureExtractor.from_pretrained(model_name_or_path)

/usr/local/lib/python3.10/dist-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


In [14]:
def transform(example_batch):
    # Take a list of PIL images and turn them to pixel values
    inputs = feature_extractor([x for x in example_batch['image']], return_tensors='pt')

    # Don't forget to include the labels!
    inputs['labels'] = example_batch['label']
    return inputs

prepared_ds = dataset.with_transform(transform)

In [15]:
ds = load_dataset("imagefolder", data_dir="Dataset_Consistent/")
prepared_ds = ds.with_transform(transform)

Resolving data files:   0%|          | 0/1203 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/401 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/402 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

In [16]:
prepared_ds

DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 1203
    })
    validation: Dataset({
        features: ['image', 'label'],
        num_rows: 401
    })
    test: Dataset({
        features: ['image', 'label'],
        num_rows: 402
    })
})

In [17]:
import numpy as np
from datasets import load_metric

def collate_fn(batch):
    return {
        'pixel_values': torch.stack([x['pixel_values'] for x in batch]),
        'labels': torch.tensor([x['labels'] for x in batch])
    }

metric = load_metric("accuracy")
def compute_metrics(p):
    return metric.compute(predictions=np.argmax(p.predictions, axis=1), references=p.label_ids)

<ipython-input-17-223a1e4a98a9>:10: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("accuracy")


In [18]:
from transformers import ViTForImageClassification

labels = prepared_ds['train'].features['label'].names

model = ViTForImageClassification.from_pretrained(
    model_name_or_path,
    num_labels=len(labels),
    id2label={str(i): c for i, c in enumerate(labels)},
    label2id={c: str(i) for i, c in enumerate(labels)}
)
NUM_CLASSES = 2
BATCH_SIZE = 16
device = 'cuda' if torch.cuda.is_available() else 'cpu'

Some weights of the model checkpoint at google/vit-large-patch16-224-in21k were not used when initializing ViTForImageClassification: ['pooler.dense.bias', 'pooler.dense.weight']
- This IS expected if you are initializing ViTForImageClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ViTForImageClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-large-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [19]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir="./vit-large",
  per_device_train_batch_size=8,
  evaluation_strategy="steps",
  num_train_epochs=4,
  fp16=True,
  save_steps=200,
  eval_steps=200,
  logging_steps=10,
  learning_rate=1e-5,
  save_total_limit=2,
  remove_unused_columns=False,
  push_to_hub=False,
  report_to='tensorboard',
  load_best_model_at_end=True,
  optim='adamw_torch',
  weight_decay=0.1
)

In [20]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=collate_fn,
    compute_metrics=compute_metrics,
    train_dataset=prepared_ds["train"],
    eval_dataset=prepared_ds["validation"],
    tokenizer=feature_extractor,
)

In [21]:
train_results = trainer.train()
trainer.save_model()
trainer.log_metrics("train", train_results.metrics)
trainer.save_metrics("train", train_results.metrics)
trainer.save_state()

Step,Training Loss,Validation Loss,Accuracy
200,0.339300,0.598591,0.675810
400,0.153100,0.598490,0.720698
600,0.047300,0.671678,0.743142


***** train metrics *****
  epoch                    =          4.0
  total_flos               = 1227642664GF
  train_loss               =       0.3085
  train_runtime            =   0:15:40.37
  train_samples_per_second =        5.117
  train_steps_per_second   =        0.642


In [22]:
metrics = trainer.evaluate(prepared_ds['validation'])
trainer.log_metrics("eval", metrics)
trainer.save_metrics("eval", metrics)

***** eval metrics *****
  epoch                   =        4.0
  eval_accuracy           =     0.7207
  eval_loss               =     0.5985
  eval_runtime            = 0:00:31.00
  eval_samples_per_second =     12.935
  eval_steps_per_second   =      1.645


In [23]:
metrics = trainer.predict(prepared_ds['test'])
trainer.log_metrics("eval", metrics.metrics)
trainer.save_metrics("eval", metrics.metrics)

***** eval metrics *****
  test_accuracy           =     0.7214
  test_loss               =     0.6147
  test_runtime            = 0:02:58.42
  test_samples_per_second =      2.253
  test_steps_per_second   =      0.286


In [25]:
model = ViTForImageClassification.from_pretrained(
    '/content/content/MyDrive/SummerResearch/vit-large/checkpoint-600',
    num_labels=len(labels),
    id2label={str(i): c for i, c in enumerate(labels)},
    label2id={c: str(i) for i, c in enumerate(labels)}
)

In [26]:
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=collate_fn,
    compute_metrics=compute_metrics,
    train_dataset=prepared_ds["train"],
    eval_dataset=prepared_ds["validation"],
    tokenizer=feature_extractor,
)

metrics = trainer.predict(prepared_ds['test'])
trainer.log_metrics("eval", metrics.metrics)
trainer.save_metrics("eval", metrics.metrics)

***** eval metrics *****
  test_accuracy           =     0.6866
  test_loss               =     0.7296
  test_runtime            = 0:00:28.92
  test_samples_per_second =     13.896
  test_steps_per_second   =      1.763
